In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [2]:
import kerastuner as kt

C:\Users\seunghyun\AppData\Local\Temp\ipykernel_16952\1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [3]:
import kerastuner as kt

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 1s 0us/step


In [5]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [6]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])

  return model

In [7]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [8]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 21s]
val_accuracy: 0.885699987411499

Best val_accuracy So Far: 0.892300009727478
Total elapsed time: 00h 14m 40s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 384 and the optimal learning rate for the optimizer
is 0.001.



In [9]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.4786 - accuracy: 0.8291 - val_loss: 0.4234 - val_accuracy: 0.8479
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3596 - accuracy: 0.8685 - val_loss: 0.3834 - val_accuracy: 0.8561
Epoch 3/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.3232 - accuracy: 0.8815 - val_loss: 0.3654 - val_accuracy: 0.8686
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2995 - accuracy: 0.8893 - val_loss: 0.3476 - val_accuracy: 0.8763
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2816 - accuracy: 0.8960 - val_loss: 0.3383 - val_accuracy: 0.8804
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2675 - accuracy: 0.9004 - val_loss: 0.3268 - val_accuracy: 0.8836
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2528 - accuracy: 0.9052 - val_loss: 0.3436 - val_accuracy: